In [54]:
import os
import json
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 2000)
pd.options.display.html.use_mathjax = False
#pd.set_option('styler.html.mathjax',False)

notebook_home = os.path.abspath('')
dataset_home = os.path.join(os.path.abspath(os.path.join(notebook_home, os.pardir)), "output")

spreadsheet_file = "Spreadsheet.csv"
template_file = "Template.csv"
function_file = "Function.csv"
reference_file = "Reference.csv"

In [55]:
s_column_names = ["s_id", "s_name", "s_dataset"]
s_df = pd.read_csv(os.path.join(dataset_home, spreadsheet_file), header=None, names=s_column_names)
s_df

,s_id,s_name,s_dataset
0,1,README.md,Enron
1,2,albert_meyers_000_1_1.pst.0.xls,Enron
2,3,albert_meyers_000_1_1.pst.1.xls,Enron
3,4,andrea_ring_000_1_1.pst.0.xls,Enron
4,5,andrea_ring_000_1_1.pst.1.xls,Enron
...,...,...,...
24918,24919,基础功能开发小组项目软件开发任务跟进表(基础功能).xlsx,GitHub
24919,24920,武汉市红十字会接受社会捐赠明细（二维码及pos机）第二期2020年1月27日.xlsx,GitHub
24920,24921,（总表）大化二中在校生疫苗接种摸底排查登记表.xlsx,GitHub
24921,24922,初中科学浙江初中科学中考真题及模拟试卷分析表2的副本.xlsx,GitHub


In [56]:
t_column_names = ["t_s_id", "t_id", "t_string", "t_func_number", "t_cell_number"]
t_df = pd.read_csv(os.path.join(dataset_home, template_file), header=None, names=t_column_names)
t_df = t_df[~t_df['t_string'].str.contains('#REF!')]
t_id_df = t_df[['t_s_id', 't_id']]
t_df

,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,20,1,A3+1,1,455
1,22,1,"DATE(YEAR(A7+31),MONTH(A7+31),1)",5,175
2,22,2,$G$3*B7,1,175
3,22,3,H36+0.01,1,134
4,22,4,"IF(E7=0,D7,E7)",2,175
...,...,...,...,...,...
102641,24923,25,"IF(DD3="""",0,DD3/80*100)",4,387
102642,24923,26,"AVERAGE(M3,Q3,U3,Y3,AC3,AG3,AK3,AO3,AS3,AW3,BA3,BE3)",1,387
102643,24923,27,"AVERAGE(BU3,BY3,CC3,CG3,CK3)",1,387
102644,24923,28,"AVERAGE(BI3,BM3,BQ3)",1,387


In [57]:
f_column_names = ["f_s_id", "f_t_id", "f_id", "f_string", "f_ref_num"]
f_df = pd.read_csv(os.path.join(dataset_home, function_file), header=None, names=f_column_names)
f_df = f_df.merge(t_id_df, left_on=['f_s_id', 'f_t_id'], right_on=['t_s_id', 't_id']).drop(columns=['t_s_id', 't_id'])
f_df

,f_s_id,f_t_id,f_id,f_string,f_ref_num
0,20,1,1,+,2
1,22,1,1,+,2
2,22,1,2,YEAR,1
3,22,1,3,+,2
4,22,1,4,MONTH,1
...,...,...,...,...,...
242795,24923,25,4,IF,3
242796,24923,26,1,AVERAGE,12
242797,24923,27,1,AVERAGE,5
242798,24923,28,1,AVERAGE,3


In [58]:
r_column_names = ["r_s_id", "r_t_id", "r_f_id", "r_id", "r_pattern_type", "r_head_row", "r_head_col", "r_tail_row", "r_tail_col", "r_scalarvalue", "r_is_interm", "r_interm_f_id"]
r_df = pd.read_csv(os.path.join(dataset_home, reference_file), header=None, names=r_column_names)
r_df = r_df.merge(t_id_df, left_on=['r_s_id', 'r_t_id'], right_on=['t_s_id', 't_id']).drop(columns=['t_s_id', 't_id'])
r_df

,r_s_id,r_t_id,r_f_id,r_id,r_pattern_type,r_head_row,r_head_col,r_tail_row,r_tail_col,r_scalarvalue,r_is_interm,r_interm_f_id
0,20,1,1,1,RR,2,0,2,0,NaN,False,-1
1,20,1,1,2,FF,0,0,0,0,1,False,-1
2,22,1,1,1,RR,6,0,6,0,NaN,False,-1
3,22,1,1,2,FF,0,0,0,0,31,False,-1
4,22,1,2,1,RR,0,0,0,0,NaN,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...
472309,24923,29,1,1,RR,2,92,2,92,NaN,False,-1
472310,24923,29,1,2,RR,4,96,4,96,NaN,False,-1
472311,24923,29,1,3,RR,2,100,2,100,NaN,False,-1
472312,24923,29,1,4,RR,2,104,2,104,NaN,False,-1


## Distribution of Spreadsheet Functions

In [59]:
df = f_df.groupby('f_string').size().reset_index(name='f_str_count')
f_dist = df.sort_values(by='f_str_count', ascending=False)
f_dist.reset_index()

,index,f_string,f_str_count
0,55,IF,32180
1,1,&,23377
2,3,+,19233
3,9,=,18502
4,4,-,18339
5,5,/,14034
6,2,*,13089
7,20,AVERAGE,11704
8,76,LOWER,11096
9,126,SUM,6101


## Preparation

In [60]:
df = r_df[r_df['r_scalarvalue'].isna()].groupby(['r_s_id', 'r_t_id', 'r_f_id']).size().reset_index(name='r_count')
df

,r_s_id,r_t_id,r_f_id,r_count
0,20,1,1,1
1,22,1,1,1
2,22,1,2,1
3,22,1,3,1
4,22,1,4,1
...,...,...,...,...
240341,24923,25,4,2
240342,24923,26,1,12
240343,24923,27,1,5
240344,24923,28,1,3


In [61]:
one_ref_df = df[df['r_count'] == 1]
n_ref_df = df[df['r_count'] > 1]
n_ref_df

,r_s_id,r_t_id,r_f_id,r_count
5,22,1,5,2
6,22,2,1,2
9,22,4,2,3
10,22,5,1,2
11,22,5,2,2
...,...,...,...,...
240341,24923,25,4,2
240342,24923,26,1,12
240343,24923,27,1,5
240344,24923,28,1,3


In [62]:
ro_df = r_df[r_df['r_scalarvalue'].isna()]
has_cell_ref_df = ro_df[(ro_df['r_head_row'] == ro_df['r_tail_row']) & (ro_df['r_head_col'] == ro_df['r_tail_col']) & (ro_df['r_pattern_type'].isin(['RR', 'FF']))][['r_s_id', 'r_t_id', 'r_f_id']].drop_duplicates()
not_all_cell_df = ro_df[(ro_df['r_head_row'] != ro_df['r_tail_row']) | (ro_df['r_head_col'] != ro_df['r_tail_col']) | (ro_df['r_pattern_type'].isin(['RF', 'FR']))][['r_s_id', 'r_t_id', 'r_f_id']].drop_duplicates()

In [63]:
has_cell_ref_df

,r_s_id,r_t_id,r_f_id
0,20,1,1
2,22,1,1
4,22,1,2
5,22,1,3
7,22,1,4
...,...,...,...
472286,24923,25,4
472289,24923,26,1
472301,24923,27,1
472306,24923,28,1


In [64]:
not_all_cell_df

,r_s_id,r_t_id,r_f_id
76,44,5,1
138,54,5,1
180,60,5,1
222,62,5,1
271,79,5,1
...,...,...,...
472040,24912,10,1
472042,24912,11,1
472044,24913,1,1
472050,24913,2,1


In [65]:
def join_t_df(df):
    return df.merge(t_df, left_on=['r_s_id', 'r_t_id'], right_on=['t_s_id', 't_id'])
    
def join_f_df(df):
    return df.merge(f_df, left_on=['r_s_id', 'r_t_id', 'r_f_id'], right_on=['f_s_id', 'f_t_id', 'f_id'])

def join_r_df(df):
    return df.merge(r_df)

def order_by_function_frequency(df):
    df = join_f_df(df)
    df = df.groupby('f_string').size().reset_index(name='f_str_count')
    return df.sort_values(by='f_str_count', ascending=False)

def select_by_t_id(df, s_id, t_id):
    return df[(df['r_s_id'] == s_id) & (df['r_t_id'] == t_id)]

def select_by_f_id(df, s_id, t_id, f_id):
    return df[(df['r_s_id'] == s_id) & (df['r_t_id'] == t_id) & (df['r_f_id'] == f_id)]

## Case One (Single Cell Reference)

In [66]:
case_one_df = one_ref_df.merge(has_cell_ref_df)[['r_s_id', 'r_t_id', 'r_f_id', 'r_count']].drop_duplicates()
case_one_df

,r_s_id,r_t_id,r_f_id,r_count
0,20,1,1,1
1,22,1,1,1
2,22,1,2,1
3,22,1,3,1
4,22,1,4,1
...,...,...,...,...
83409,24923,22,3,1
83410,24923,24,2,1
83411,24923,24,3,1
83412,24923,25,2,1


In [67]:
case_one_enrich_df = join_t_df(case_one_df)
case_one_enrich_df

,r_s_id,r_t_id,r_f_id,r_count,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,20,1,1,1,20,1,A3+1,1,455
1,22,1,1,1,22,1,"DATE(YEAR(A7+31),MONTH(A7+31),1)",5,175
2,22,1,2,1,22,1,"DATE(YEAR(A7+31),MONTH(A7+31),1)",5,175
3,22,1,3,1,22,1,"DATE(YEAR(A7+31),MONTH(A7+31),1)",5,175
4,22,1,4,1,22,1,"DATE(YEAR(A7+31),MONTH(A7+31),1)",5,175
...,...,...,...,...,...,...,...,...,...
83409,24923,22,3,1,24923,22,"IF(CR3="""",0,CR3/95*100)",4,387
83410,24923,24,2,1,24923,24,"IF(CZ3="""",0,CZ3/95*100)",4,387
83411,24923,24,3,1,24923,24,"IF(CZ3="""",0,CZ3/95*100)",4,387
83412,24923,25,2,1,24923,25,"IF(DD3="""",0,DD3/80*100)",4,387


In [68]:
case_one_enrich_df[['t_string']].drop_duplicates()

,t_string
0,A3+1
1,"DATE(YEAR(A7+31),MONTH(A7+31),1)"
5,H36+0.01
6,"IF(E7=0,D7,E7)"
7,+Y6-X6
...,...
83404,"IF(CJ3="""",0,CJ3/80*100)"
83406,"IF(CN3="""",0,CN3/85*100)"
83408,"IF(CR3="""",0,CR3/95*100)"
83410,"IF(CZ3="""",0,CZ3/95*100)"


In [69]:
order_by_function_frequency(case_one_df)

,f_string,f_str_count
1,&,16340
9,=,12324
59,LOWER,11096
3,+,4913
51,ISNUMBER,4352
42,IF,3274
4,-,3221
2,*,3074
54,LEN,2907
5,/,1971


## Case Three (Single Range Reference)

In [70]:
case_three_df = one_ref_df.merge(not_all_cell_df)[['r_s_id', 'r_t_id', 'r_f_id']].drop_duplicates()
case_three_df

,r_s_id,r_t_id,r_f_id
0,44,5,1
1,54,5,1
2,60,5,1
3,62,5,1
4,79,5,1
...,...,...,...
28965,24871,2,1
28966,24890,1,1
28967,24903,1,1
28968,24903,2,1


In [71]:
case_three_enrich_df = join_t_df(case_three_df)
case_three_enrich_df

,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,44,5,1,44,5,E11*SUM(P11:Q11),2,9990
1,54,5,1,54,5,E11*SUM(P11:Q11),2,9990
2,60,5,1,60,5,E11*SUM(P11:Q11),2,9990
3,62,5,1,62,5,E11*SUM(P11:Q11),2,9990
4,79,5,1,79,5,E11*SUM(P11:Q11),2,9990
...,...,...,...,...,...,...,...,...
28965,24871,2,1,24871,2,SUM(D6:AY6),1,112
28966,24890,1,1,24890,1,LEN(D:D),1,502
28967,24903,1,1,24903,1,SUM(H3:S3),1,7806
28968,24903,2,1,24903,2,SUM(H3:S3),1,7806


In [72]:
order_by_function_frequency(case_three_df)

,f_string,f_str_count
8,AVERAGE,11290
25,MAX,5299
27,MIN,5171
38,SUM,5132
15,COUNTIF,899
6,ABS,300
20,LARGE,176
12,COUNT,128
36,STDEV,72
18,ISNA,67


In [73]:
df = join_f_df(case_three_df)
df[df['f_string'] == '*']

,r_s_id,r_t_id,r_f_id,f_s_id,f_t_id,f_id,f_string,f_ref_num
2193,19043,1,1,19043,1,1,*,2


In [74]:
select_by_t_id(case_three_enrich_df, 19043, 1)

,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
2193,19043,1,1,19043,1,1000*B1:B2639,1,2639


In [75]:
df = join_r_df(case_three_df)
res_df = df[(df['r_head_col'] != df['r_tail_col']) & ((df['r_head_row'] != df['r_tail_row']) | (df['r_pattern_type'].isin(['RF', 'FR'])))]
order_by_function_frequency(res_df)

,f_string,f_str_count
4,AVERAGE,40
9,MIN,31
3,=,12
10,SUM,9
1,+,4
8,MAX,3
6,HLOOKUP,2
7,ISNUMBER,2
0,&,1
2,-,1


In [76]:
res_df = df[(df['r_head_row'] != df['r_tail_row'])]
order_by_function_frequency(res_df)

,f_string,f_str_count
5,AVERAGE,338
22,SUM,231
21,STDEV,63
12,ISNUMBER,56
7,COUNT,54
17,MAX,46
19,MIN,42
18,MEDIAN,30
24,UPPER,26
4,=,22


In [77]:
res_df = join_t_df(res_df[['r_s_id', 'r_t_id', 'r_f_id']])
#res_df[res_df['t_string'].str.contains('=')
res_df[res_df['r_s_id'] == 18598]
#res_df[res_df['f_string'].str.contains('=')]

,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
237,18598,10,2,18598,10,"IF(OFFSET($X2,-1,0)=TRUE,TRUE,FALSE)",3,9999
238,18598,12,2,18598,12,"IF(OFFSET($Z2,-1,0)=TRUE,TRUE,FALSE)",3,9999
239,18598,14,2,18598,14,"IF(OFFSET($AB2,-1,0)=TRUE,TRUE,FALSE)",3,9999
240,18598,16,2,18598,16,"IF(OFFSET($AD2,-1,0)=TRUE,TRUE,FALSE)",3,9999
241,18598,18,2,18598,18,"IF(OFFSET($AF2,-1,0)=TRUE,TRUE,FALSE)",3,9999
242,18598,30,2,18598,30,"IF(OFFSET($AD2,-1,0)=TRUE,TRUE,FALSE)",3,9999
243,18598,32,2,18598,32,"IF(OFFSET($AF2,-1,0)=TRUE,TRUE,FALSE)",3,9999
244,18598,34,2,18598,34,"IF(OFFSET($AH2,-1,0)=TRUE,TRUE,FALSE)",3,9999
245,18598,36,2,18598,36,"IF(OFFSET($AJ2,-1,0)=TRUE,TRUE,FALSE)",3,9999
246,18598,38,2,18598,38,"IF(OFFSET($AL2,-1,0)=TRUE,TRUE,FALSE)",3,9999


## Case Four (Multiple Cell/Range References)

In [78]:
case_four_df = n_ref_df.merge(not_all_cell_df)[['r_s_id', 'r_t_id', 'r_f_id']].drop_duplicates()
case_four_df

,r_s_id,r_t_id,r_f_id
0,469,1,5
1,469,2,5
2,469,3,5
3,469,4,5
4,469,5,5
...,...,...,...
4060,24912,10,1
4061,24912,11,1
4062,24913,1,1
4063,24913,2,1


In [79]:
case_four_enrich_df = join_f_df(case_four_df)
case_four_enrich_df

,r_s_id,r_t_id,r_f_id,f_s_id,f_t_id,f_id,f_string,f_ref_num
0,469,1,5,469,1,5,INDEX,3
1,469,2,5,469,2,5,INDEX,3
2,469,3,5,469,3,5,INDEX,3
3,469,4,5,469,4,5,INDEX,3
4,469,5,5,469,5,5,INDEX,3
...,...,...,...,...,...,...,...,...
4060,24912,10,1,24912,10,1,INDEX,2
4061,24912,11,1,24912,11,1,INDEX,2
4062,24913,1,1,24913,1,1,COUNTIF,2
4063,24913,2,1,24913,2,1,MATCH,3


In [80]:
vlookup_df = case_four_enrich_df[case_four_enrich_df['f_string'] == 'VLOOKUP']
df = f_df[f_df['f_string'] == 'VLOOKUP']
df = df.merge(vlookup_df, left_on=['f_s_id', 'f_t_id', 'f_id'], right_on=['r_s_id', 'r_t_id', 'r_f_id'], how='left', indicator=True)
df[df['_merge'] == 'left_only']

,f_s_id_x,f_t_id_x,f_id_x,f_string_x,f_ref_num_x,r_s_id,r_t_id,r_f_id,f_s_id_y,f_t_id_y,f_id_y,f_string_y,f_ref_num_y,_merge
1326,24173,12,5,VLOOKUP,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [81]:
t_df[(t_df['t_s_id'] == 24173) & (t_df['t_id'] == 12)]

,t_s_id,t_id,t_string,t_func_number,t_cell_number
98178,24173,12,"VLOOKUP(I1,IF(G1=5,$O$14:$P$17,IF(G1=4,$O$9:$P$12,$O$4:$P$7)),2,0)",5,332


In [82]:
order_by_function_frequency(case_four_df)

,f_string,f_str_count
36,VLOOKUP,1357
19,INDEX,978
33,SUMIF,424
13,COUNTIF,229
29,RANK,210
17,HLOOKUP,173
23,MATCH,165
18,IF,158
32,SUM,70
34,SUMPRODUCT,60


In [102]:
res_df = join_r_df(join_f_df(case_four_df))
res_df1 = res_df[(res_df['f_string'] == 'IF')].reset_index()
res_df1 = res_df1[['r_s_id', 'r_t_id', 'r_f_id']]
join_t_df(res_df1).drop_duplicates().reset_index()

,index,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,0,16953,1,10,16953,1,"IF(AND(OFFSET(A60,-$B$4,0,1,1)>=$A$34,ISNUMBER(OFFSET(A60,-$B$4,0,1,1))),OFFSET(A60,-$B$4,0,1,1),"""")",10,340
1,3,16953,8,5,16953,8,"IF(L61="""","""",OFFSET(L61,MAX($K$34:K60)-K61,-1,1,1))",5,339
2,6,18121,1,5,18121,1,"SUMIF(IF(MID($D167,1,1)=""A"",$C$27:$C$51,IF(MID($D167,1,1)=""B"",$D$27:$D$51,$E$27:$E$51)),$C167,$I$27:$I$51)",7,385
3,9,18121,1,6,18121,1,"SUMIF(IF(MID($D167,1,1)=""A"",$C$27:$C$51,IF(MID($D167,1,1)=""B"",$D$27:$D$51,$E$27:$E$51)),$C167,$I$27:$I$51)",7,385
4,12,18121,2,5,18121,2,"SUMIF(IF(MID($D167,2,1)=""A"",$C$55:$C$79,IF(MID($D167,2,1)=""B"",$D$55:$D$79,$E$55:$E$79)),$C167,$I$55:$I$79)",7,385
5,15,18121,2,6,18121,2,"SUMIF(IF(MID($D167,2,1)=""A"",$C$55:$C$79,IF(MID($D167,2,1)=""B"",$D$55:$D$79,$E$55:$E$79)),$C167,$I$55:$I$79)",7,385
6,18,18121,3,8,18121,3,"IF(MID($D167,3,1)=""@"",SUM($I$83:$I$107),SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),$C167,$I$83:$I$107)+IF($C167=""BAR"",0,SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),""BAR"",$I$83:$I$107)))",21,385
7,21,18121,3,9,18121,3,"IF(MID($D167,3,1)=""@"",SUM($I$83:$I$107),SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),$C167,$I$83:$I$107)+IF($C167=""BAR"",0,SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),""BAR"",$I$83:$I$107)))",21,385
8,24,18121,3,16,18121,3,"IF(MID($D167,3,1)=""@"",SUM($I$83:$I$107),SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),$C167,$I$83:$I$107)+IF($C167=""BAR"",0,SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),""BAR"",$I$83:$I$107)))",21,385
9,27,18121,3,17,18121,3,"IF(MID($D167,3,1)=""@"",SUM($I$83:$I$107),SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),$C167,$I$83:$I$107)+IF($C167=""BAR"",0,SUMIF(IF(MID($D167,3,1)=""A"",$C$83:$C$107,IF(MID($D167,3,1)=""B"",$D$83:$D$107,$E$83:$E$107)),""BAR"",$I$83:$I$107)))",21,385


In [84]:
res_df = join_r_df(join_f_df(case_four_df))
res_df1 = res_df[(res_df['f_string'] == 'VLOOKUP') & (((res_df['r_id'] == 3) & (res_df['r_pattern_type'] != 'FF')) | ((res_df['r_id'] == 2) & (res_df['r_pattern_type'] != 'FF')))].reset_index()
res_df1

,index,r_s_id,r_t_id,r_f_id,f_s_id,f_t_id,f_id,f_string,f_ref_num,r_id,r_pattern_type,r_head_row,r_head_col,r_tail_row,r_tail_col,r_scalarvalue,r_is_interm,r_interm_f_id
0,1538,17620,1,3,17620,1,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
1,1542,17620,2,3,17620,2,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
2,1546,17620,3,3,17620,3,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
3,1550,17620,4,3,17620,4,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
4,1554,17620,5,3,17620,5,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
5,1558,17620,6,3,17620,6,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
6,1562,17620,7,3,17620,7,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
7,1566,17620,8,3,17620,8,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
8,1570,17620,9,3,17620,9,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2
9,1574,17620,10,3,17620,10,3,VLOOKUP,4,2,RR,0,0,1,1,NaN,True,2


In [85]:
res_df1 = res_df1[['r_s_id', 'r_t_id', 'r_f_id']]
join_t_df(res_df1).drop_duplicates().reset_index()

,index,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,0,17620,1,3,17620,1,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),2,FALSE)",3,421
1,1,17620,2,3,17620,2,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),3,FALSE)",3,421
2,2,17620,3,3,17620,3,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),4,FALSE)",3,421
3,3,17620,4,3,17620,4,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),5,FALSE)",3,421
4,4,17620,5,3,17620,5,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),6,FALSE)",3,421
5,5,17620,6,3,17620,6,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),7,FALSE)",3,421
6,6,17620,7,3,17620,7,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),8,FALSE)",3,421
7,7,17620,8,3,17620,8,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),9,FALSE)",3,421
8,8,17620,9,3,17620,9,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),10,FALSE)",3,421
9,9,17620,10,3,17620,10,"VLOOKUP($A2,INDIRECT($D2&""!$A$1:$Z$300""),11,FALSE)",3,421


In [86]:
res_df = join_r_df(join_f_df(case_four_df))
res_df1 = res_df[(res_df['f_string'] == 'HLOOKUP') & (((res_df['r_id'] == 3) & (res_df['r_pattern_type'] != 'FF')) | ((res_df['r_id'] == 2) & (res_df['r_pattern_type'] != 'FF')))].reset_index()
res_df1

,index,r_s_id,r_t_id,r_f_id,f_s_id,f_t_id,f_id,f_string,f_ref_num,r_id,r_pattern_type,r_head_row,r_head_col,r_tail_row,r_tail_col,r_scalarvalue,r_is_interm,r_interm_f_id
0,1418,17458,10,4,17458,10,4,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
1,1419,17458,10,4,17458,10,4,HLOOKUP,4,3,RR,4,0,4,0,NaN,False,-1
2,1422,17458,11,3,17458,11,3,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
3,1423,17458,11,3,17458,11,3,HLOOKUP,4,3,RR,4,0,4,0,NaN,False,-1
4,1426,17458,11,4,17458,11,4,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
5,1427,17458,11,4,17458,11,4,HLOOKUP,4,3,RR,4,0,4,0,NaN,False,-1
6,1430,17458,12,4,17458,12,4,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
7,1434,17458,13,4,17458,13,4,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
8,1438,17458,14,5,17458,14,5,HLOOKUP,4,2,FR,0,123,4,138,NaN,False,-1
9,1439,17458,14,5,17458,14,5,HLOOKUP,4,3,RR,4,0,4,0,NaN,False,-1


In [87]:
res_df1 = res_df1[['r_s_id', 'r_t_id', 'r_f_id']]
join_t_df(res_df1).drop_duplicates().reset_index()

,index,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,0,17458,10,4,17458,10,"IF(F5-1<0.0001,0,HLOOKUP($F5-1,$DT$1:$EI5,$A5,FALSE))",5,2167
1,2,17458,11,3,17458,11,"IF(F5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,$A5,FALSE),HLOOKUP($F5,$DT$1:$EI5,$A5,FALSE))",5,2167
2,4,17458,11,4,17458,11,"IF(F5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,$A5,FALSE),HLOOKUP($F5,$DT$1:$EI5,$A5,FALSE))",5,2167
3,6,17458,12,4,17458,12,"IF(H5-1<0.0001,0.5,HLOOKUP($F5-1,$DT$1:$EI5,3,FALSE))",5,2167
4,7,17458,13,4,17458,13,"IF(H5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,2,FALSE),HLOOKUP($F5,$DT$1:$EI5,2,FALSE))",5,2167
5,8,17458,14,5,17458,14,"IF(F5=G5,H5,IF(G5-1<0.0001,0,HLOOKUP($G5-1,$DT$1:$EI5,$A5,FALSE)))",7,2167
6,10,17458,15,4,17458,15,"IF(F5=G5,I5,IF(G5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,$A5,FALSE),HLOOKUP($G5,$DT$1:$EI5,$A5,FALSE)))",7,2167
7,12,17458,15,5,17458,15,"IF(F5=G5,I5,IF(G5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,$A5,FALSE),HLOOKUP($G5,$DT$1:$EI5,$A5,FALSE)))",7,2167
8,14,17458,16,5,17458,16,"IF(F5=G5,J5,IF(I5-1<0.0001,0.5,HLOOKUP($G5-1,$DT$1:$EI5,3,FALSE)))",7,2167
9,15,17458,17,5,17458,17,"IF(F5=G5,K5,IF(I5-1<0.0001,HLOOKUP(1,$DT$1:$EI5,2,FALSE),HLOOKUP($G5,$DT$1:$EI5,2,FALSE)))",7,2167


In [88]:
res_df = join_r_df(join_f_df(case_four_df))
res_df1 = res_df[(res_df['f_string'] == 'LOOKUP')].reset_index()
res_df1

,index,r_s_id,r_t_id,r_f_id,f_s_id,f_t_id,f_id,f_string,f_ref_num,r_id,r_pattern_type,r_head_row,r_head_col,r_tail_row,r_tail_col,r_scalarvalue,r_is_interm,r_interm_f_id
0,1357,17429,1,1,17429,1,1,LOOKUP,3,1,RR,1,6,1,6,NaN,False,-1
1,1358,17429,1,1,17429,1,1,LOOKUP,3,2,FF,0,0,1048575,0,NaN,False,-1
2,1359,17429,1,1,17429,1,1,LOOKUP,3,3,FF,0,1,1048575,1,NaN,False,-1
3,1360,17429,2,1,17429,2,1,LOOKUP,3,1,RR,1,6,1,6,NaN,False,-1
4,1361,17429,2,1,17429,2,1,LOOKUP,3,2,FF,0,0,1048575,0,NaN,False,-1
5,1362,17429,2,1,17429,2,1,LOOKUP,3,3,FF,0,3,1048575,3,NaN,False,-1
6,1363,17429,3,1,17429,3,1,LOOKUP,3,1,RR,1,6,1,6,NaN,False,-1
7,1364,17429,3,1,17429,3,1,LOOKUP,3,2,FF,0,0,1048575,0,NaN,False,-1
8,1365,17429,3,1,17429,3,1,LOOKUP,3,3,FF,0,2,1048575,2,NaN,False,-1
9,1366,17429,4,1,17429,4,1,LOOKUP,3,1,RR,1,6,1,6,NaN,False,-1


In [89]:
res_df1 = res_df1[['r_s_id', 'r_t_id', 'r_f_id']]
join_t_df(res_df1).drop_duplicates().reset_index()

,index,r_s_id,r_t_id,r_f_id,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,0,17429,1,1,17429,1,"LOOKUP(G2,A:A,B:B)",1,3224
1,3,17429,2,1,17429,2,"LOOKUP(G2,A:A,D:D)",1,3224
2,6,17429,3,1,17429,3,"LOOKUP(G2,A:A,C:C)",1,3224
3,9,17429,4,1,17429,4,"LOOKUP(G2,A:A,E:E)",1,3224
4,12,17429,5,1,17429,5,"LOOKUP(G2,A$1:A$1048576,B$1:B$1048576)",1,3119
5,15,17429,7,1,17429,7,"LOOKUP(G2,A:A,C:C)",1,3119
6,18,17429,8,1,17429,8,"LOOKUP(G2,A:A,E:E)",1,3119
7,21,17429,9,1,17429,9,"LOOKUP(E2,A:A,B:B)",1,3224
8,24,17429,10,1,17429,10,"LOOKUP(E2,A:A,C:C)",1,3224
9,27,22480,7,1,22480,7,"LOOKUP(P2,$C$2:$C$145,$L$2:$L$145)",1,2880


## Case Two (Multiple Cell References)

In [90]:
case_two_df = n_ref_df.merge(case_four_df, how='left', indicator=True)
case_two_df = case_two_df[case_two_df['_merge'] == 'left_only'].drop(columns=['r_count', '_merge'])
case_two_df

,r_s_id,r_t_id,r_f_id
0,22,1,5
1,22,2,1
2,22,4,2
3,22,5,1
4,22,5,2
...,...,...,...
127957,24923,25,4
127958,24923,26,1
127959,24923,27,1
127960,24923,28,1


In [91]:
order_by_function_frequency(case_two_df)

,f_string,f_str_count
28,IF,28748
3,-,15097
2,+,14302
4,/,11960
1,*,9963
0,&,7016
8,=,6106
7,<>,5696
12,AND,4431
50,SEARCH,3170


In [92]:
case_two_diff_row_df = join_r_df(case_two_df).groupby(['r_s_id', 'r_t_id', 'r_f_id'])['r_head_row'].nunique().reset_index(name='r_row_count')
case_two_diff_row_df = case_two_diff_row_df[case_two_diff_row_df['r_row_count'] > 1]
case_two_diff_row_df

,r_s_id,r_t_id,r_f_id,r_row_count
1,22,2,1,2
2,22,4,2,2
4,22,5,2,2
6,22,7,1,2
7,22,8,2,2
...,...,...,...,...
123887,24923,23,1,2
123888,24923,23,2,2
123889,24923,24,1,2
123891,24923,25,1,2


In [93]:
case_two_diff_col_df = join_r_df(case_two_df).groupby(['r_s_id', 'r_t_id', 'r_f_id'])['r_head_col'].nunique().reset_index(name='r_col_count')
case_two_diff_col_df = case_two_diff_col_df[case_two_diff_col_df['r_col_count'] > 1]
case_two_diff_col_df

,r_s_id,r_t_id,r_f_id,r_col_count
1,22,2,1,2
2,22,4,2,3
3,22,5,1,2
4,22,5,2,2
5,22,6,1,2
...,...,...,...,...
123891,24923,25,1,2
123893,24923,26,1,12
123894,24923,27,1,5
123895,24923,28,1,3


In [94]:
order_by_function_frequency(case_two_diff_col_df)

,f_string,f_str_count
3,-,13378
4,/,10938
27,IF,10906
2,+,10366
1,*,9294
0,&,6345
7,<>,5601
8,=,4935
9,>,2196
47,SEARCH,2066


In [95]:
join_t_df(case_two_diff_col_df)

,r_s_id,r_t_id,r_f_id,r_col_count,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,22,2,1,2,22,2,$G$3*B7,1,175
1,22,4,2,3,22,4,"IF(E7=0,D7,E7)",2,175
2,22,5,1,2,22,5,F7*G7*K7,2,175
3,22,5,2,2,22,5,F7*G7*K7,2,175
4,22,6,1,2,22,6,F7*G7,1,175
...,...,...,...,...,...,...,...,...,...
85189,24923,25,1,2,24923,25,"IF(DD3="""",0,DD3/80*100)",4,387
85190,24923,26,1,12,24923,26,"AVERAGE(M3,Q3,U3,Y3,AC3,AG3,AK3,AO3,AS3,AW3,BA3,BE3)",1,387
85191,24923,27,1,5,24923,27,"AVERAGE(BU3,BY3,CC3,CG3,CK3)",1,387
85192,24923,28,1,3,24923,28,"AVERAGE(BI3,BM3,BQ3)",1,387


In [96]:
case_two_range_df = case_two_diff_row_df.merge(case_two_diff_col_df)
case_two_range_df

,r_s_id,r_t_id,r_f_id,r_row_count,r_col_count
0,22,2,1,2,2
1,22,4,2,2,3
2,22,5,2,2,2
3,22,7,1,2,2
4,22,8,2,2,2
...,...,...,...,...,...
55995,24923,23,1,2,2
55996,24923,23,2,2,2
55997,24923,24,1,2,2
55998,24923,25,1,2,2


In [97]:
order_by_function_frequency(case_two_range_df)

,f_string,f_str_count
24,IF,10846
2,+,6523
0,&,5965
7,<>,5575
1,*,4869
8,=,4503
3,-,3488
4,/,2494
43,SEARCH,2065
9,>,2009


In [98]:
join_t_df(case_two_range_df)

,r_s_id,r_t_id,r_f_id,r_row_count,r_col_count,t_s_id,t_id,t_string,t_func_number,t_cell_number
0,22,2,1,2,2,22,2,$G$3*B7,1,175
1,22,4,2,2,3,22,4,"IF(E7=0,D7,E7)",2,175
2,22,5,2,2,2,22,5,F7*G7*K7,2,175
3,22,7,1,2,2,22,7,X6*$Z$3,1,151
4,22,8,2,2,2,22,8,+Y6-X6,2,151
...,...,...,...,...,...,...,...,...,...,...
55995,24923,23,1,2,2,24923,23,"IF(CV3="""",0,CV3)",2,387
55996,24923,23,2,2,2,24923,23,"IF(CV3="""",0,CV3)",2,387
55997,24923,24,1,2,2,24923,24,"IF(CZ3="""",0,CZ3/95*100)",4,387
55998,24923,25,1,2,2,24923,25,"IF(DD3="""",0,DD3/80*100)",4,387
